# DS-SF-23 | Demo 14 | Latent Variable Modeling with `gensim`

`gensim` (http://radimrehurek.com/gensim) is a library of language processing tools focused on latent variable models of text.

In [3]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from gensim import matutils, models

pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

%matplotlib inline
plt.style.use('ggplot')

ImportError: No module named 'gensim'

The data is about sentiments on Amazon reviews.

In [4]:
reviews = []
sentiments = []

with open(os.path.join('..', 'datasets', 'amazon-reviews.txt')) as f:
    for line in f.readlines():
        line = line.strip('\n')
        review, sentiment = line.split('\t')
        sentiment = np.nan if sentiment == '' else int(sentiment)

        reviews.append(review.lower())
        sentiments.append(sentiment)

df = pd.DataFrame({'review': reviews, 'sentiment': sentiments})

In [5]:
df.head()

,review,sentiment
0,i try not to adjust the volume setting to avoi...,NaN
1,so there is no way for me to plug it in here i...,0.0
2,"good case, excellent value.",1.0
3,i thought motorola made reliable products!.,NaN
4,battery for motorola razr.,NaN


In [6]:
df.dropna(inplace = True) # Let's drop the NaN

In [7]:
df.head()

,review,sentiment
1,so there is no way for me to plug it in here i...,0.0
2,"good case, excellent value.",1.0
5,great for the jawbone.,1.0
10,tied to charger for conversations lasting more...,0.0
11,the mic is great.,1.0


## LDA with `gensim`

### Let's first translate a set of documents (articles) into a matrix representation with a row per document and a column per feature (word or n-gram)

In [24]:
vectorizer = feature_extraction.text.CountVectorizer(stop_words = 'english')

In [25]:
documents = vectorizer.fit_transform(df.review)

In [26]:
# Let's now build a mapping of numerical ID to word

id2word = dict(enumerate(vectorizer.get_feature_names()))

In [11]:
id2word

{0: '10',
 1: '100',
 2: '11',
 3: '12',
 4: '13',
 5: '15',
 6: '15g',
 7: '18',
 8: '20',
 9: '2000',
 10: '2005',
 11: '2160',
 12: '24',
 13: '2mp',
 14: '325',
 15: '350',
 16: '375',
 17: '3o',
 18: '42',
 19: '44',
 20: '45',
 21: '4s',
 22: '50',
 23: '5020',
 24: '510',
 25: '5320',
 26: '680',
 27: '700w',
 28: '8125',
 29: '8525',
 30: '8530',
 31: 'abhor',
 32: 'ability',
 33: 'able',
 34: 'abound',
 35: 'absolutel',
 36: 'absolutely',
 37: 'ac',
 38: 'accept',
 39: 'acceptable',
 40: 'access',
 41: 'accessable',
 42: 'accessing',
 43: 'accessory',
 44: 'accessoryone',
 45: 'accidentally',
 46: 'accompanied',
 47: 'according',
 48: 'activate',
 49: 'activated',
 50: 'activesync',
 51: 'actually',
 52: 'ad',
 53: 'adapter',
 54: 'adapters',
 55: 'add',
 56: 'addition',
 57: 'additional',
 58: 'address',
 59: 'adhesive',
 60: 'adorable',
 61: 'advertised',
 62: 'advise',
 63: 'aggravating',
 64: 'ago',
 65: 'alarm',
 66: 'allot',
 67: 'allow',
 68: 'allowing',
 69: 'allows',


### We want to learn which columns are correlated (i.e., likely to come from the same topic).  This is the word distribution.  We can also determine what topics are in each document, the topic distribution.

In [12]:
# First we convert our word-matrix into gensim's format

corpus = matutils.Sparse2Corpus(documents, documents_columns = False)

NameError: name 'matutils' is not defined

(Check https://radimrehurek.com/gensim/matutils as needed)

In [13]:
corpus

NameError: name 'corpus' is not defined

(Check https://radimrehurek.com/gensim/models/ldamodel as needed)

In [27]:
# Then we fit an LDA model using the models object from gensim and its ldamodel class

model = models.ldamodel.LdaModel(corpus = corpus, num_topics = 25, id2word = id2word, passes = 10)

NameError: name 'models' is not defined

In this model, we need to explicitly specify the number of topic we want the model to uncover.  This is a critical parameter, but there isn't much guidance on how to choose it.  Try to use domain expertise where possible.

In [15]:
model

NameError: name 'model' is not defined

### Goodness of fit

Now we need to assess the goodness of fit for our model.  Like other unsupervised learning techniques, our validation techniques are mostly about interpretation.

Use the following questions to guide you:
- Did we learn reasonable topics?
- Do the words that make up a topic make sense?
- Is this topic helpful towards our goal?

In [16]:
model.print_topics()

NameError: name 'model' is not defined

Some topics will be clearer than others.  The following topics represent clear concepts:
- Cooking and Recipes: 0.009 \* cup + 0.009 \* recipe + 0.007 \* make + 0.007 \* food + 0.006 \* sugar
- Cooking and recipes: 0.013 \* butter + 0.010 \* baking + 0.010 \* dough + 0.009 \* cup + 0.009 \* sugar
- Fashion and Style: 0.013 \* fashion + 0.006 \* like + 0.006 \* dress + 0.005 \* style

## Word2Vec with `gensim`

In [17]:
# Setup the body text
sentences = df.review.map(lambda review: review.split())

In [18]:
sentences

1       [so, there, is, no, way, for, me, to, plug, it...
2                        [good, case,, excellent, value.]
5                             [great, for, the, jawbone.]
10      [tied, to, charger, for, conversations, lastin...
11                                 [the, mic, is, great.]
13      [i, have, to, jiggle, the, plug, to, get, it, ...
15      [if, you, have, several, dozen, or, several, h...
17      [if, you, are, razr, owner...you, must, have, ...
18            [needless, to, say,, i, wasted, my, money.]
21               [what, a, waste, of, money, and, time!.]
24                 [and, the, sound, quality, is, great.]
26      [he, was, very, impressed, when, going, from, ...
29      [if, the, two, were, seperated, by, a, mere, 5...
32                          [very, good, quality, though]
33      [the, design, is, very, odd,, as, the, ear, "c...
34      [highly, recommend, for, any, one, who, has, a...
36            [i, advise, everyone, do, not, be, fooled!]
39            

In [19]:
model = models.Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)

NameError: name 'models' is not defined

`Word2Vec` has many arguments:
- `size` represents how many concepts or topics we should use
- `window` represents how many words surrounding a sentence we should use as our original feature
- `min_count` is the number of times that context or word must appear
- `workers` is the number of CPU cores to use to speed up model training

(Check http://radimrehurek.com/gensim/models/word2vec as needed)

In [20]:
model

NameError: name 'model' is not defined

### Most similar words

The model has a `most_similar` function that helps find the words most similar to the one you queried.  This will return words that are most often used in the same context.

In [21]:
model.most_similar(positive = ['great'])

NameError: name 'model' is not defined

In [22]:
vectorizer.get_feature_names()

['10',
 '100',
 '11',
 '12',
 '13',
 '15',
 '15g',
 '18',
 '20',
 '2000',
 '2005',
 '2160',
 '24',
 '2mp',
 '325',
 '350',
 '375',
 '3o',
 '42',
 '44',
 '45',
 '4s',
 '50',
 '5020',
 '510',
 '5320',
 '680',
 '700w',
 '8125',
 '8525',
 '8530',
 'abhor',
 'ability',
 'able',
 'abound',
 'absolutel',
 'absolutely',
 'ac',
 'accept',
 'acceptable',
 'access',
 'accessable',
 'accessing',
 'accessory',
 'accessoryone',
 'accidentally',
 'accompanied',
 'according',
 'activate',
 'activated',
 'activesync',
 'actually',
 'ad',
 'adapter',
 'adapters',
 'add',
 'addition',
 'additional',
 'address',
 'adhesive',
 'adorable',
 'advertised',
 'advise',
 'aggravating',
 'ago',
 'alarm',
 'allot',
 'allow',
 'allowing',
 'allows',
 'alot',
 'aluminum',
 'amazed',
 'amazing',
 'amazon',
 'amp',
 'ample',
 'angeles',
 'angle',
 'answer',
 'ant',
 'antena',
 'anti',
 'apart',
 'apartment',
 'apparently',
 'appealing',
 'appearance',
 'appears',
 'applifies',
 'appointments',
 'area',
 'arguing',
 'a

In [23]:
sentences

1       [so, there, is, no, way, for, me, to, plug, it...
2                        [good, case,, excellent, value.]
5                             [great, for, the, jawbone.]
10      [tied, to, charger, for, conversations, lastin...
11                                 [the, mic, is, great.]
13      [i, have, to, jiggle, the, plug, to, get, it, ...
15      [if, you, have, several, dozen, or, several, h...
17      [if, you, are, razr, owner...you, must, have, ...
18            [needless, to, say,, i, wasted, my, money.]
21               [what, a, waste, of, money, and, time!.]
24                 [and, the, sound, quality, is, great.]
26      [he, was, very, impressed, when, going, from, ...
29      [if, the, two, were, seperated, by, a, mere, 5...
32                          [very, good, quality, though]
33      [the, design, is, very, odd,, as, the, ear, "c...
34      [highly, recommend, for, any, one, who, has, a...
36            [i, advise, everyone, do, not, be, fooled!]
39            

In [31]:
# remove all stop words (e.g., 'the')
sentences = list(map(lambda sentence:
                     list(filter(lambda word: word in vectorizer.get_feature_names(), sentence)), sentences))

In [32]:
sentences

[['way', 'plug', 'unless'],
 ['good', 'excellent'],
 ['great'],
 ['tied', 'charger', 'conversations', 'lasting', '45'],
 ['mic'],
 ['jiggle', 'plug', 'line', 'right', 'decent'],
 ['dozen', 'imagine', 'fun', 'sending'],
 ['razr'],
 ['needless', 'wasted'],
 ['waste', 'money'],
 ['sound', 'quality'],
 ['impressed', 'going', 'original', 'battery', 'extended'],
 ['seperated',
  'mere',
  'ft',
  'started',
  'notice',
  'excessive',
  'static',
  'garbled',
  'sound'],
 ['good', 'quality'],
 ['design', 'ear', 'comfortable'],
 ['highly', 'recommend', 'blue', 'tooth'],
 ['advise'],
 ['far'],
 ['works'],
 ['clicks', 'place', 'way', 'makes', 'wonder', 'long', 'mechanism'],
 ['went', 'website', 'followed', 'pair'],
 ['bought', 'use', 'kindle', 'absolutely', 'loved'],
 ['commercials'],
 ['run', 'new', 'battery', 'bars', 'days'],
 ['bought', 'mother', 'problem'],
 ['great', 'pocket', 'pc', 'phone'],
 ['owned', 'phone', 'months', 'say', 'best', 'mobile', 'phone'],
 ['think', 'instructions', 'provid

In [30]:
model = models.Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)

NameError: name 'models' is not defined

In [33]:
model.most_similar(positive = ['great'])

NameError: name 'model' is not defined